In [1]:
# default_exp to_metaflow

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# export


import asyncio
import multiprocessing
import os
import subprocess
import sys
from itertools import product
from pathlib import Path, PosixPath
from typing import Any, Dict, Iterable

import pandas as pd
from fastcore.script import Param, call_parse
from nbdev.export import find_default_export, get_config, nbglob, read_nb

from sciflow.data_handler import extract_param_meta
from sciflow.params import params_as_dict
from sciflow.parse_module import FuncDetails, extract_steps
from sciflow.utils import get_flow_path, prepare_env

# Sciflow Notebook to MetaFlow Flow

> Converts from a `sciflow` format notebook to a `metaflow` flow. 

Supported features:

* Linear/sequential DAGs
* Simple `Parameters`

In [4]:
nb_path = Path(os.path.join("test", "test_export.ipynb"))
nb = read_nb(nb_path)
module_name = find_default_export(nb["cells"]).replace(".", "/")
test_module = os.path.join(get_config().path("lib_path"), f"{module_name}.py")

In [5]:
# export


def titleize(name):
    return name.title().replace("_", "")

In [6]:
assert titleize("snake_case") == "SnakeCase"

In [7]:
# export


def rename_steps_for_metaflow(steps):
    for i, step in enumerate(steps):
        if i == 0:
            step.name = "start"

In [8]:
steps = extract_steps(test_module)

In [9]:
no_steps = extract_steps(os.path.join(get_config().path("lib_path"), f"_nbdev.py"))
assert len(no_steps) == 0

In [10]:
assert ["first", "preprocess", "train", "last"] == [step.name for step in steps]
rename_steps_for_metaflow(steps)
assert ["start", "preprocess", "train", "last"] == [step.name for step in steps]

In [11]:
# export


def indent_multiline(multiline_text, indent=1):
    lines = multiline_text.strip().split("\n")
    spaces = "".join(["    " for _ in range(indent)])
    for i in range(len(lines)):
        prefix = spaces if i > 0 else spaces + '"""'
        lines[i] = prefix + lines[i]
    return "\n".join(lines) + '"""'

In [12]:
text = """
Some text
:param param: text
"""
assert '    """Some text\n    :param param: text"""' == indent_multiline(text)

In [13]:
prepare_env()

In [14]:
nb_path = Path(os.path.join("test", "test_multistep.ipynb"))
nb = read_nb(nb_path)
module_name = find_default_export(nb["cells"]).replace(".", "/")
test_module = os.path.join(get_config().path("lib_path"), f"{module_name}.py")
steps = extract_steps(test_module)

In [15]:
# export


def nb_to_metaflow(nb_path: Path, flow_path: Path, silent=True, track_experiment=True):
    nb = read_nb(nb_path)
    lib_name = get_config().get("lib_name")
    module_name = find_default_export(nb["cells"])
    if not module_name:
        return
    module_name = module_name
    path_sep_module_name = module_name.replace(".", "/")
    nb_name = os.path.basename(nb_path)
    exported_module = os.path.join(
        get_config().path("lib_path"), f"{path_sep_module_name}.py"
    )
    steps = extract_steps(exported_module)
    if len(steps) == 0:
        return
    orig_step_names = [step.name for step in steps]
    if len(steps) == 1:
        steps.append(FuncDetails("end", None, None, False, "", "pass"))
    params = params_as_dict(nb_path)
    if len(params) == 0:
        print(f"No params cell found for: {os.path.basename(nb_path)}")
    flow_class_name = f"{titleize(extract_module_only(module_name))}Flow"
    rename_steps_for_metaflow(steps)
    write_module_to_file(
        flow_path,
        flow_class_name,
        lib_name,
        module_name,
        orig_step_names,
        steps,
        params,
        track_experiment,
    )
    if not silent:
        print(
            f"Converted {nb_name} to {flow_class_name} in: {os.path.basename(flow_path)}"
        )

In [16]:
pkg_module_name = "test.module"
module_name = "module"
path_sep_module_name = module_name.replace(".", "/")

In [17]:
# export


def extract_module_only(package_module_name):
    module_name = package_module_name
    if "." in module_name:
        package_name, module_name = module_name.split(".")
    return module_name

In [18]:
assert "module" == extract_module_only(module_name)
assert "module" == extract_module_only(pkg_module_name)

In [19]:
#  export


def write_module_to_file(
    flow_path: Path,
    flow_class_name: str,
    lib_name: str,
    module_name: str,
    orig_step_names: Iterable[str],
    steps: Iterable[FuncDetails],
    params: dict,
    track_experiment: bool,
):
    if not os.path.exists(flow_path.parent):
        os.mkdir(flow_path.parent)
    fq_module_name = f"{lib_name}.{module_name}"
    param_meta = extract_param_meta(fq_module_name, params)
    with open(flow_path, "w") as flow_file:
        flow_file.write("#!/usr/bin/env python\n")
        flow_file.write("# coding=utf-8\n")
        flow_file.write("# SCIFLOW GENERATED FILE - EDIT COMPANION NOTEBOOK\n")
        has_mf_param = any((p.has_metaflow_param for p in param_meta.values()))
        has_json_param = any((p.is_json_type for p in param_meta.values()))
        mf_params_import = "from metaflow import FlowSpec, step, current"
        if has_mf_param:
            mf_params_import += ", Parameter"
        if has_json_param:
            mf_params_import += ", JSONType"
            flow_file.write("import json\n")
        flow_file.write(mf_params_import + "\n")
        flow_file.write(f"from {fq_module_name} import {', '.join(orig_step_names)}\n")
        if len(params) > 0:
            flow_file.write(
                f"from {fq_module_name} import {', '.join(params.keys())}\n"
            )

        if track_experiment:
            flow_file.write("from sacred import Experiment\n")
            flow_file.write(
                "from sciflow.experiment.lake_observer import AWSLakeObserver\n"
            )
            flow_file.write("import time")
            write_observers(
                lib_name,
                flow_file,
                module_name,
                os.environ["SCIFLOW_BUCKET"],
                get_config().get("lib_name"),
            )

        flow_file.write(f"\n\nclass {flow_class_name}(FlowSpec):\n")
        single_indent = "    "
        write_params(flow_file, param_meta, single_indent)
        if track_experiment:
            flow_file.write(f"{single_indent}artifacts = []\n")
            flow_file.write(f"{single_indent}metrics = []\n")
        flow_file.write("\n")
        write_steps(
            flow_file,
            steps,
            orig_step_names,
            param_meta,
            single_indent,
            track_experiment,
        )
        write_track_flow(flow_file, track_experiment)
        flow_file.write("\n")

        flow_file.write('if __name__ == "__main__":\n')
        flow_file.write(f"{single_indent}{flow_class_name}()")

In [20]:
# export


def write_observers(lib_name, flow_file, module_name, bucket_name, project):
    experiment_name = extract_module_only(module_name)
    sacred_setup = f"""

ex = Experiment("{experiment_name}")
# TODO inject observers
obs = AWSLakeObserver(project="{lib_name}", experiment_name="{experiment_name}")
ex.observers.append(obs)

@ex.config
def config():
    flow_run_id = None
    artifacts = []
    metrics = []
    """
    flow_file.write(sacred_setup)

In [21]:
# export


def write_track_flow(flow_file, track_experiment):
    track_flow = """
    @step
    def end(self):
        flow_info = {
            "flow_name": current.flow_name,
            "run_id": current.run_id,
            "pathspec": current.pathspec,
            "namespace": current.namespace,
            "username": current.username,
            "flow_parameters": str(current.parameter_names),
            "run_time_mins": round((time.time() - self.__getattr__('start_time')) / 60.0, 1)
        }
    
        run = ex.run(config_updates={'artifacts': self.__getattr__('artifacts'),
                                    'metrics': self.__getattr__('metrics')},
                     meta_info = flow_info)
        
    @ex.main
    def track_flow(artifacts, metrics, _run):
        for artifact in artifacts:
            _run.add_artifact(artifact)
        for metric_name, metric_value, step in metrics:
            _run.log_scalar(metric_name, metric_value, step)
    """
    if not track_experiment:
        track_flow = """
    @step
    def end(self):
        pass
    """
    flow_file.write(track_flow)

In [22]:
#  export


def write_params(flow_file, param_meta, single_indent):
    for param in param_meta.keys():
        if param_meta[param].is_scalar:
            flow_file.write(
                f"{single_indent}{param} = Parameter('{param}', default={param})\n"
            )
        elif param_meta[param].is_json_type:
            flow_file.write(
                f"{single_indent}{param} = Parameter('{param}', default=json.dumps({param}), type=JSONType)\n"
            )
        elif param_meta[param].instance_type == PosixPath:
            flow_file.write(
                f"{single_indent}{param} = Parameter('{param}', default=str({param}))\n"
            )

In [23]:
nb_path = os.path.join(Path(".").resolve(), "test", "test_data_handling.ipynb")
params = params_as_dict(nb_path)
param_meta = extract_param_meta("sciflow.test.test_data_handling", params)

In [24]:
assert any((p.has_metaflow_param for p in param_meta.values()))
assert any((p.is_json_type for p in param_meta.values()))

In [25]:
# export


def format_arg(arg, param_meta):
    if arg in param_meta and not param_meta[arg].has_metaflow_param:
        result = arg
    else:
        result = "self." + arg
    return result


def write_steps(
    flow_file, steps, orig_step_names, param_meta, single_indent, track_experiment
):
    for i, step in enumerate(steps):
        flow_file.write(f"{single_indent}@step\n")
        flow_file.write(f"{single_indent}def {step.name}(self):\n")
        if step.docstring:
            flow_file.write(f"{indent_multiline(step.docstring, 2)}\n")
        # Check for padded step
        if i < len(orig_step_names):
            flow_step_args = ""
            if len(step.args) > 0:
                flow_step_args = ", ".join(
                    [format_arg(a, param_meta) for a in step.args.split(",")]
                )
            if not step.has_return:
                flow_file.write(
                    f"{single_indent}{single_indent}{orig_step_names[i]}({flow_step_args})\n"
                )
            else:
                if step.return_stmt in param_meta:
                    raise ValueError(
                        f"[{os.path.basename(flow_file.name)}] step return variable {step.return_stmt} shadows a parameter name - parameters must be unique"
                    )
                flow_file.write(
                    f"{single_indent}{single_indent}results = {orig_step_names[i]}({flow_step_args})\n"
                )
                write_track_capture(flow_file)
        if i == 0 and track_experiment:
            flow_file.write(
                f"{single_indent}{single_indent}self.start_time = time.time()\n"
            )
        if i < len(steps):
            next_step = "end" if i == len(steps) - 1 else steps[i + 1].name
            flow_file.write(
                f"{single_indent}{single_indent}self.next(self.{next_step})\n"
            )
        flow_file.write("\n")

In [26]:
# export


def write_track_capture(flow_file):
    flow_file.write(
        f"""
        for key in results.keys():
            if key in self.__dict__:
                self.__dict__[key] = self.__dict__[key] + results[key]
            else:
                self.__dict__[key] = results[key]

"""
    )

# Test Flow Generation

In [29]:
assert Path(
    Path(".").resolve(), "test", "flows", "metaflow", f"test_data_handling.py"
) == get_flow_path(Path(Path(".").resolve(), "test", f"test_data_handling.ipynb"))

In [30]:
nb_to_metaflow(nb_path, get_flow_path(nb_path), silent=False, track_experiment=True)

Converted test_data_handling.ipynb to TestDataHandlingFlow in: test_data_handling.py


In [31]:
nb_to_metaflow(
    Path(
        Path(".").resolve(),
        "test",
        "test_multistep.ipynb",
    ),
    get_flow_path(Path(Path(".").resolve(), "test", "test_multistep.ipynb")),
    silent=False,
    track_experiment=True,
)

Converted test_multistep.ipynb to TestMultistepFlow in: test_multistep.py


# Ignore notebooks without Sciflow steps

In [32]:
nb_to_metaflow("packaging.ipynb", get_flow_path("packaging.ipynb"), silent=False)

In [33]:
# export


def generate_flows(config=None, track_experiment=True):
    nb_paths = nbglob(recursive=True)
    for nb_path in nb_paths:
        nb_to_metaflow(
            nb_path,
            get_flow_path(nb_path, config=config),
            track_experiment=track_experiment,
            silent=False,
        )

In [34]:
generate_flows()

Converted test_multistep.ipynb to TestMultistepFlow in: test_multistep.py
Converted test_export.ipynb to TestExportFlow in: test_export.py
Converted test_module.ipynb to TestModuleFlow in: test_module.py
No params cell found for: test_multistep_no_params.ipynb
Converted test_multistep_no_params.ipynb to TestMultistepNoParamsFlow in: test_multistep_no_params.py
Converted test_data_handling.ipynb to TestDataHandlingFlow in: test_data_handling.py


In [35]:
generate_flows(track_experiment=False)

Converted test_multistep.ipynb to TestMultistepFlow in: test_multistep.py
Converted test_export.ipynb to TestExportFlow in: test_export.py
Converted test_module.ipynb to TestModuleFlow in: test_module.py
No params cell found for: test_multistep_no_params.ipynb
Converted test_multistep_no_params.ipynb to TestMultistepNoParamsFlow in: test_multistep_no_params.py
Converted test_data_handling.ipynb to TestDataHandlingFlow in: test_data_handling.py


In [36]:
# export


@call_parse
def sciflow_metaflow(track: Param("Track flows as sacred experiments", bool) = True):
    generate_flows(get_config(), track)